In [1]:
from matplotlib import pyplot as plt
from sklearn import cluster, preprocessing
from shapely.geometry import Polygon, Point
import shapely.wkt
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings, cv2
warnings.filterwarnings('ignore')

In [2]:
def getPointCoords(row, geom, coord_type):
    exterior = row[geom]
    if coord_type == 'x':
        return list(exterior.coords.xy[0])[0]
    elif coord_type == 'y':
        return list(exterior.coords.xy[1])[0]

def getPolyCoords(row, geom, coord_type):
    exterior = row[geom].exterior
    if coord_type == 'x':
        return list(exterior.coords.xy[0])
    elif coord_type == 'y':
        return list(exterior.coords.xy[1])

def load_useful():
    def load_data():
        path = '/home/herry/hackthon/data/sq_2km_center0519.csv'
        return pd.read_csv(path).rename(columns={'wkt_geom': 'geometry'})
    
    def to_gdf(df):
        geom = df['geometry'].apply(lambda x: shapely.wkt.loads(x))
        crs = {'init': 'epsg:3857'}
        return gpd.GeoDataFrame(df, crs=crs, geometry=geom)
    
    def main():
        gdf = to_gdf(load_data())
        gdf['lng'] = gdf.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
        gdf['lat'] = gdf.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)
        return gdf
    
    return main()

In [3]:
def load_image(path, rgb=False):
    img = cv2.imread(path)
    if rgb:
        b, g, r = cv2.split(img)
        img = cv2.merge([r, g, b])
        
    return img

def contour_of_specific_color(image, color):
    mask = np.where((image[:, :, 0] == color[0]) &
                    (image[:, :, 1] == color[1]) &
                    (image[:, :, 2] == color[2]), 1, 0)
    mask = mask.astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
    return contours

def to_geopandas(contours):
    def getPolyCoords(row, geom):
        exterior = row[geom].exterior
        return [(x, y) for x, y in zip(exterior.coords.xy[0], exterior.coords.xy[1])]

    def reverse_picture(polygon_coor):
        return [(point[0], 1180 - point[1]) for point in polygon_coor]

    def list_to_hexagon(polygon_coor):
        return Polygon(polygon_coor)
    
    Polygon_l = [Polygon(c[:, 0, :]) for c in contours if c.shape[0] > 2]
    data = {'id': [i for i in range(len(Polygon_l))]}
    crs = {'init' :'epsg:3857'}
    geometry = gpd.GeoSeries(Polygon_l)
    gdf = gpd.GeoDataFrame(data, crs=crs, geometry=geometry)

    gdf['coor'] = gdf.apply(getPolyCoords, geom='geometry', axis=1)
    gdf['reverse_coor'] = gdf['coor'].apply(reverse_picture)
    gdf['geometry'] = gdf['reverse_coor'].apply(list_to_hexagon)
    return gdf

def convert_index_to_coordinate(row, center, coord_type=None):
    length = 1000
    bottom_left = (center[0] - length, center[1] - length)
    top_right = (center[0] + length, center[1] + length)
    x_max = 1180
    y_max = 1180

    exterior = row['geometry'].exterior
    l = []
    
    if coord_type == 'x':
        for x in exterior.coords.xy[0]:
            l.append(bottom_left[0] + (top_right[0] - bottom_left[0]) * x / x_max)
    
    elif coord_type == 'y':
        for y in exterior.coords.xy[1]:
            l.append(bottom_left[1] + (top_right[1] - bottom_left[1]) * y / y_max)

    return l

def change_geometry(gdf):
    l = []
    for x, y in zip(gdf['lng'], gdf['lat']):
        l.append([x, y])
    return Polygon(l)

In [4]:
gdf_useful = load_useful()

In [5]:
gdf_l = []
i = 0
for image_id, lng, lat in gdf_useful[['uiid', 'lng', 'lat']].values:
    path = f'/home/herry/hackthon/data/label/output_{int(image_id)}.png'
    img = load_image(path, rgb=True)

    df_rgb = pd.DataFrame(img.reshape(img.shape[0]*img.shape[1], 3))
    df_rgb['count'] = 1
    df_static = df_rgb.groupby([0, 1, 2])['count'].sum().reset_index().sort_values(['count'])
    df_static = df_static[df_static['count'] >= 500]

    for r, g, b, _ in df_static.values:
        contours = contour_of_specific_color(img, color=(r, g, b))
        try:
            gdf = to_geopandas(contours)
            gdf['image_id'] = image_id
            gdf['r'] = r
            gdf['g'] = g
            gdf['b'] = b
            
            center = (lng, lat)
            gdf['lng'] = gdf.apply(convert_index_to_coordinate, center=center, coord_type='x', axis=1)
            gdf['lat'] = gdf.apply(convert_index_to_coordinate, center=center, coord_type='y', axis=1)
            gdf['geometry'] = gdf.apply(change_geometry, axis=1)
            gdf = gdf.to_crs(epsg=4326)
            gdf_l.append(gdf)
        except:
            print(r, g, b, ' does not work')
    print(i, len(gdf_useful))
    i += 1
gdf = pd.concat(gdf_l)

0 175
1 175
2 175
3 175
4 175
5 175
6 175
7 175
8 175
9 175
10 175
11 175
12 175
13 175
14 175
15 175
16 175
17 175
18 175
19 175
20 175
21 175
22 175
23 175
24 175
25 175
26 175
27 175
28 175
29 175
30 175
31 175
32 175
33 175
34 175
35 175
36 175
37 175
38 175
39 175
40 175
41 175
42 175
43 175
44 175
45 175
46 175
47 175
48 175
49 175
50 175
51 175
52 175
53 175
54 175
55 175
56 175
57 175
58 175
59 175
60 175
61 175
62 175
63 175
64 175
65 175
66 175
67 175
68 175
69 175
70 175
71 175
72 175
73 175
74 175
75 175
76 175
77 175
78 175
79 175
80 175
81 175
82 175
83 175
84 175
85 175
86 175
87 175
88 175
89 175
90 175
91 175
92 175
93 175
94 175
95 175
96 175
97 175
98 175
99 175
100 175
101 175
102 175
103 175
104 175
105 175
106 175
107 175
108 175
109 175
110 175
111 175
112 175
113 175
114 175
115 175
116 175
117 175
118 175
119 175
120 175
121 175
122 175
123 175
124 175
125 175
126 175
127 175
128 175
129 175
130 175
131 175
132 175
133 175
134 175
135 175
136 175
137 175
138 17

In [8]:
gdf

,id,geometry,coor,reverse_coor,image_id,r,g,b,lng,lat
0,0,"POLYGON ((121.53933 25.08348, 121.53933 25.083...","[(1001.0, 1179.0), (1001.0, 1180.0), (1004.0, ...","[(1001.0, 1.0), (1001.0, 0.0), (1004.0, 0.0), ...",118.0,1.0,1.0,1.0,"[13529696.61016949, 13529696.61016949, 1352970...","[2886001.694915254, 2886000.0, 2886000.0, 2886..."
1,1,"POLYGON ((121.53915 25.08348, 121.53913 25.083...","[(989.0, 1179.0), (988.0, 1180.0), (987.0, 118...","[(989.0, 1.0), (988.0, 0.0), (987.0, 0.0), (99...",118.0,1.0,1.0,1.0,"[13529676.271186441, 13529674.576271186, 13529...","[2886001.694915254, 2886000.0, 2886000.0, 2886..."
2,2,"POLYGON ((121.53824 25.08348, 121.53824 25.083...","[(929.0, 1179.0), (929.0, 1180.0), (933.0, 118...","[(929.0, 1.0), (929.0, 0.0), (933.0, 0.0), (93...",118.0,1.0,1.0,1.0,"[13529574.576271186, 13529574.576271186, 13529...","[2886001.694915254, 2886000.0, 2886000.0, 2886..."
3,3,"POLYGON ((121.53569 25.08348, 121.53569 25.083...","[(762.0, 1179.0), (762.0, 1180.0), (763.0, 118...","[(762.0, 1.0), (762.0, 0.0), (763.0, 0.0), (76...",118.0,1.0,1.0,1.0,"[13529291.525423728, 13529291.525423728, 13529...","[2886001.694915254, 2886000.0, 2886000.0, 2886..."
4,4,"POLYGON ((121.54148 25.08351, 121.54148 25.083...","[(1142.0, 1177.0), (1142.0, 1178.0), (1143.0, ...","[(1142.0, 3.0), (1142.0, 2.0), (1143.0, 2.0), ...",118.0,1.0,1.0,1.0,"[13529935.593220338, 13529935.593220338, 13529...","[2886005.084745763, 2886003.3898305083, 288600..."
...,...,...,...,...,...,...,...,...,...,...
7243,7243,"POLYGON ((121.53268 24.92062, 121.53269 24.920...","[(564.0, 1.0), (565.0, 0.0), (566.0, 1.0), (56...","[(564.0, 1179.0), (565.0, 1180.0), (566.0, 117...",107.0,0.0,0.0,0.0,"[13528955.93220339, 13528957.627118643, 135289...","[2865998.305084746, 2866000.0, 2865998.3050847..."
7244,7244,"POLYGON ((121.52953 24.92062, 121.52954 24.920...","[(357.0, 1.0), (358.0, 0.0), (359.0, 1.0), (35...","[(357.0, 1179.0), (358.0, 1180.0), (359.0, 117...",107.0,0.0,0.0,0.0,"[13528605.084745763, 13528606.779661017, 13528...","[2865998.305084746, 2866000.0, 2865998.3050847..."
7245,7245,"POLYGON ((121.52916 24.92062, 121.52918 24.920...","[(333.0, 1.0), (334.0, 0.0), (335.0, 1.0), (33...","[(333.0, 1179.0), (334.0, 1180.0), (335.0, 117...",107.0,0.0,0.0,0.0,"[13528564.406779662, 13528566.101694915, 13528...","[2865998.305084746, 2866000.0, 2865998.3050847..."
7246,7246,"POLYGON ((121.52901 24.92062, 121.52902 24.920...","[(323.0, 1.0), (324.0, 0.0), (325.0, 1.0), (32...","[(323.0, 1179.0), (324.0, 1180.0), (325.0, 117...",107.0,0.0,0.0,0.0,"[13528547.45762712, 13528549.152542373, 135285...","[2865998.305084746, 2866000.0, 2865998.3050847..."


In [9]:
cols = ['image_id', 'id', 'geometry', 'r', 'g', 'b']
path = '/home/herry/hackthon/data/building_kmean.csv'
gdf[cols].to_csv(path, index=False)

In [11]:
# gdf['lng'] = gdf.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
# gdf['lat'] = gdf.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [17]:
# from bokeh.io import show
# from bokeh.models import ColumnDataSource, GMapOptions, HoverTool, CustomJS
# from bokeh.models import LogColorMapper, LinearColorMapper
# from bokeh.models import ColorBar, NumeralTickFormatter
# from bokeh.palettes import RdYlGn11 as palette
# from bokeh.plotting import gmap, figure
# from bokeh.io import output_notebook, output_file
# from bokeh.layouts import column, gridplot
# output_notebook()

In [18]:
# map_options = GMapOptions(lng=121.4, lat=24.96, map_type="satellite", zoom=13)
# source = ColumnDataSource(data=dict(gdf.drop(columns=['geometry'])))
# p = gmap(API(), map_options, title="Taiwan", width=1500, height=1000)
# p.patches('lng', 'lat', source=source, fill_alpha=1, line_color="black", line_width=0.2, legend_label='網格')
# p.legend.click_policy="hide"
# show(p)